In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow
import missingno as msno
import os


import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

# Modling:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Model Evaluation:
from sklearn.metrics import silhouette_score

# IN CASE you don't have pyarrow  and plotlyinstalled, run the following command in your terminal
# pip install pyarrow 
# pip install plotly

In [2]:
df = pd.read_parquet('daily_20221004.parquet.gzip')

In [3]:
df['uid'] = df['symbol'] + '_'+ df['id'].astype(str)

In [4]:
df_t = df.copy()
df_t.head()

time_open  \
symbol_id time_close                                           
MPH_7742  2020-11-23T00:59:59.999Z  2020-11-23T00:00:00.000Z   
          2020-11-24T00:59:59.999Z  2020-11-24T00:00:00.000Z   
          2020-11-25T00:59:59.999Z  2020-11-25T00:00:00.000Z   
          2020-11-26T00:59:59.999Z  2020-11-26T00:00:00.000Z   
          2020-11-27T00:59:59.999Z  2020-11-27T00:00:00.000Z   

                                                   time_high  \
symbol_id time_close                                           
MPH_7742  2020-11-23T00:59:59.999Z  2020-11-23T00:00:07.000Z   
          2020-11-24T00:59:59.999Z  2020-11-24T00:00:07.000Z   
          2020-11-25T00:59:59.999Z  2020-11-25T00:00:06.000Z   
          2020-11-26T00:59:59.999Z  2020-11-26T00:56:05.000Z   
          2020-11-27T00:59:59.999Z  2020-11-27T00:57:05.000Z   

                                                    time_low  quote.USD.open  \
symbol_id time_close                                                           
MPH_7742  2020-11-23T00:59:59.999Z  2020-11-23T00:37:05.000Z       90.554933   
          2020-11-24T00:59:59.999Z  2020-11-24T00:40:06.000Z       85.161030   
          2020-11-25T00:59:59.999Z  2020-11-25T00:58:05.000Z       57.527366   
          2020-11-26T00:59:59.999Z  2020-11-26T00:13:06.000Z       39.466839   
          2020-11-27T00:59:59.999Z  2020-11-27T00:16:05.000Z       30.002092   

                                    quote.USD.high  quote.USD.low  \
symbol_id time_close                                                
MPH_7742  2020-11-23T00:59:59.999Z       90.554933      79.852330   
          2020-11-24T00:59:59.999Z       85.161030      80.694133   
          2020-11-25T00:59:59.999Z       57.527366      53.679986   
          2020-11-26T00:59:59.999Z       40.121814      38.980499   
          2020-11-27T00:59:59.999Z       30.391962      29.690305   

                                    quote.USD.close  quote.USD.volume  \
symbol_id time_close                                                    
MPH_7742  2020-11-23T00:59:59.999Z        85.599205      1.600851e+07   
          2020-11-24T00:59:59.999Z        81.645754      1.009738e+07   
          2020-11-25T00:59:59.999Z        53.704963      8.899399e+06   
          2020-11-26T00:59:59.999Z        40.056399      4.225065e+06   
          2020-11-27T00:59:59.999Z        30.383005      2.933446e+06   

                                    quote.USD.market_cap  \
symbol_id time_close                                       
MPH_7742  2020-11-23T00:59:59.999Z                   0.0   
          2020-11-24T00:59:59.999Z                   0.0   
          2020-11-25T00:59:59.999Z                   0.0   
          2020-11-26T00:59:59.999Z                   0.0   
          2020-11-27T00:59:59.999Z                   0.0   

                                         quote.USD.timestamp symbol    id  \
symbol_id time_close                                                        
MPH_7742  2020-11-23T00:59:59.999Z  2020-11-23T00:59:59.999Z    MPH  7742   
          2020-11-24T00:59:59.999Z  2020-11-24T00:59:59.999Z    MPH  7742   
          2020-11-25T00:59:59.999Z  2020-11-25T00:59:59.999Z    MPH  7742   
          2020-11-26T00:59:59.999Z  2020-11-26T00:59:59.999Z    MPH  7742   
          2020-11-27T00:59:59.999Z  2020-11-27T00:59:59.999Z    MPH  7742   

                                         uid  
symbol_id time_close                          
MPH_7742  2020-11-23T00:59:59.999Z  MPH_7742  
          2020-11-24T00:59:59.999Z  MPH_7742  
          2020-11-25T00:59:59.999Z  MPH_7742  
          2020-11-26T00:59:59.999Z  MPH_7742  
          2020-11-27T00:59:59.999Z  MPH_7742

In [5]:
df_t.columns.to_series().groupby(df_t.dtypes).groups

{int64: ['id'], float64: ['quote.USD.open', 'quote.USD.high', 'quote.USD.low', 'quote.USD.close', 'quote.USD.volume', 'quote.USD.market_cap'], object: ['time_open', 'time_high', 'time_low', 'quote.USD.timestamp', 'symbol', 'uid']}

In [6]:
features = ['quote.USD.open',
       'quote.USD.high', 'quote.USD.low', 'quote.USD.close',
       'quote.USD.volume', 'quote.USD.market_cap']
df_t = df_t[features]
scaler = StandardScaler()
scaler.fit(df_t)
print(scaler.mean_)
df_n = scaler.transform(df_t)
df_n = pd.DataFrame(df_n, columns = features)
df_n.head()

[2.59362676e+02 2.61223981e+02 2.57357434e+02 2.59305516e+02
 1.21227236e+10 1.23850265e+09]


,quote.USD.open,quote.USD.high,quote.USD.low,quote.USD.close,quote.USD.volume,quote.USD.market_cap
0,-0.051759,-0.051956,-0.054870,-0.053279,-0.002106,-0.05942
1,-0.053413,-0.053598,-0.054610,-0.054491,-0.002107,-0.05942
2,-0.061886,-0.062011,-0.062961,-0.063061,-0.002107,-0.05942
3,-0.067423,-0.067309,-0.067504,-0.067248,-0.002108,-0.05942
4,-0.070325,-0.070271,-0.070376,-0.070215,-0.002108,-0.05942


In [7]:
df_final = pd.concat([df.reset_index().drop(['quote.USD.open',
       'quote.USD.high', 'quote.USD.low', 'quote.USD.close',
       'quote.USD.volume', 'quote.USD.market_cap'], axis =1),df_n], axis = 1)
df_final.head()

,symbol_id,time_close,time_open,time_high,time_low,quote.USD.timestamp,symbol,id,uid,quote.USD.open,quote.USD.high,quote.USD.low,quote.USD.close,quote.USD.volume,quote.USD.market_cap
0,MPH_7742,2020-11-23T00:59:59.999Z,2020-11-23T00:00:00.000Z,2020-11-23T00:00:07.000Z,2020-11-23T00:37:05.000Z,2020-11-23T00:59:59.999Z,MPH,7742,MPH_7742,-0.051759,-0.051956,-0.054870,-0.053279,-0.002106,-0.05942
1,MPH_7742,2020-11-24T00:59:59.999Z,2020-11-24T00:00:00.000Z,2020-11-24T00:00:07.000Z,2020-11-24T00:40:06.000Z,2020-11-24T00:59:59.999Z,MPH,7742,MPH_7742,-0.053413,-0.053598,-0.054610,-0.054491,-0.002107,-0.05942
2,MPH_7742,2020-11-25T00:59:59.999Z,2020-11-25T00:00:00.000Z,2020-11-25T00:00:06.000Z,2020-11-25T00:58:05.000Z,2020-11-25T00:59:59.999Z,MPH,7742,MPH_7742,-0.061886,-0.062011,-0.062961,-0.063061,-0.002107,-0.05942
3,MPH_7742,2020-11-26T00:59:59.999Z,2020-11-26T00:00:00.000Z,2020-11-26T00:56:05.000Z,2020-11-26T00:13:06.000Z,2020-11-26T00:59:59.999Z,MPH,7742,MPH_7742,-0.067423,-0.067309,-0.067504,-0.067248,-0.002108,-0.05942
4,MPH_7742,2020-11-27T00:59:59.999Z,2020-11-27T00:00:00.000Z,2020-11-27T00:57:05.000Z,2020-11-27T00:16:05.000Z,2020-11-27T00:59:59.999Z,MPH,7742,MPH_7742,-0.070325,-0.070271,-0.070376,-0.070215,-0.002108,-0.05942


In [8]:
df_final = df_final.set_index(['symbol_id', 'time_close'])

In [9]:
df_final = df_final.drop(['id','symbol'], axis = 1)

In [10]:
df_final = df_final.drop(['time_open', 'quote.USD.timestamp'], axis = 1)

In [11]:
df_final['minute_high'] = [t.minute for t in pd.DatetimeIndex(df.time_high)]
df_final['minute_low'] = [t.minute for t in pd.DatetimeIndex(df.time_low)]
df_final['sec_high'] = [t.second for t in pd.DatetimeIndex(df.time_high)]
df_final['sec_low'] = [t.second for t in pd.DatetimeIndex(df.time_low)]

In [12]:
df_final = df_final.drop(['time_high', 'time_low'], axis = 1)

In [13]:
new_feature = ['minute_high', 'minute_low', 'sec_high', 'sec_low']
df_newcat = df_final[new_feature]
df_newcat = scaler.fit_transform(df_newcat)
df_newcat = pd.DataFrame(df_newcat, columns=new_feature)

In [14]:
df_final_copy = df_final.copy()
df_final_copy = pd.concat([df_final_copy.reset_index().drop(new_feature, axis = 1), df_newcat], axis = 1)
df_final_copy = df_final_copy.set_index(['symbol_id', 'time_close'])

In [15]:
df_final_copy = df_final_copy.dropna(axis = 0)

In [16]:
df_final_copy 

uid  quote.USD.open  quote.USD.high  \
symbol_id time_close                                                           
MPH_7742  2020-11-23T00:59:59.999Z  MPH_7742       -0.051759       -0.051956   
          2020-11-24T00:59:59.999Z  MPH_7742       -0.053413       -0.053598   
          2020-11-25T00:59:59.999Z  MPH_7742       -0.061886       -0.062011   
          2020-11-26T00:59:59.999Z  MPH_7742       -0.067423       -0.067309   
          2020-11-27T00:59:59.999Z  MPH_7742       -0.070325       -0.070271   
...                                      ...             ...             ...   
CXO_2490  2022-07-01T00:59:59.999Z  CXO_2490       -0.079504       -0.079503   
          2022-07-02T00:59:59.999Z  CXO_2490       -0.079505       -0.079504   
          2022-07-03T00:59:59.999Z  CXO_2490       -0.079505       -0.079504   
          2022-07-04T00:59:59.999Z  CXO_2490       -0.079505       -0.079504   
          2022-07-05T00:59:59.999Z  CXO_2490       -0.079503       -0.079503   

                                    quote.USD.low  quote.USD.close  \
symbol_id time_close                                                 
MPH_7742  2020-11-23T00:59:59.999Z      -0.054870        -0.053279   
          2020-11-24T00:59:59.999Z      -0.054610        -0.054491   
          2020-11-25T00:59:59.999Z      -0.062961        -0.063061   
          2020-11-26T00:59:59.999Z      -0.067504        -0.067248   
          2020-11-27T00:59:59.999Z      -0.070376        -0.070215   
...                                           ...              ...   
CXO_2490  2022-07-01T00:59:59.999Z      -0.079534        -0.079513   
          2022-07-02T00:59:59.999Z      -0.079534        -0.079514   
          2022-07-03T00:59:59.999Z      -0.079534        -0.079514   
          2022-07-04T00:59:59.999Z      -0.079534        -0.079514   
          2022-07-05T00:59:59.999Z      -0.079533        -0.079513   

                                    quote.USD.volume  quote.USD.market_cap  \
symbol_id time_close                                                         
MPH_7742  2020-11-23T00:59:59.999Z         -0.002106             -0.059420   
          2020-11-24T00:59:59.999Z         -0.002107             -0.059420   
          2020-11-25T00:59:59.999Z         -0.002107             -0.059420   
          2020-11-26T00:59:59.999Z         -0.002108             -0.059420   
          2020-11-27T00:59:59.999Z         -0.002108             -0.059420   
...                                              ...                   ...   
CXO_2490  2022-07-01T00:59:59.999Z         -0.002108             -0.058753   
          2022-07-02T00:59:59.999Z         -0.002108             -0.058784   
          2022-07-03T00:59:59.999Z         -0.002108             -0.058783   
          2022-07-04T00:59:59.999Z         -0.002108             -0.058780   
          2022-07-05T00:59:59.999Z         -0.002108             -0.058738   

                                    minute_high  minute_low  sec_high  \
symbol_id time_close                                                    
MPH_7742  2020-11-23T00:59:59.999Z    -1.308405    0.420403 -0.056313   
          2020-11-24T00:59:59.999Z    -1.308405    0.564676 -0.056313   
          2020-11-25T00:59:59.999Z    -1.308405    1.430313 -0.167048   
          2020-11-26T00:59:59.999Z     1.395706   -0.733778 -0.277782   
          2020-11-27T00:59:59.999Z     1.443994   -0.589506 -0.277782   
...                                         ...         ...       ...   
CXO_2490  2022-07-01T00:59:59.999Z     0.333377   -1.022324 -0.831453   
          2022-07-02T00:59:59.999Z    -0.342651    1.478404 -0.831453   
          2022-07-03T00:59:59.999Z    -0.487514    0.324222 -0.831453   
          2022-07-04T00:59:59.999Z    -0.535802   -1.022324 -0.831453   
          2022-07-05T00:59:59.999Z     1.347418   -1.022324 -0.831453   

                                     sec_low  
symbol_id time_close                          
MPH_7742  2020-11-23T00:59:59.

In [17]:
df_final_copy.drop('uid', inplace=True, axis=1)
df_final_copy

quote.USD.open  quote.USD.high  \
symbol_id time_close                                                 
MPH_7742  2020-11-23T00:59:59.999Z       -0.051759       -0.051956   
          2020-11-24T00:59:59.999Z       -0.053413       -0.053598   
          2020-11-25T00:59:59.999Z       -0.061886       -0.062011   
          2020-11-26T00:59:59.999Z       -0.067423       -0.067309   
          2020-11-27T00:59:59.999Z       -0.070325       -0.070271   
...                                            ...             ...   
CXO_2490  2022-07-01T00:59:59.999Z       -0.079504       -0.079503   
          2022-07-02T00:59:59.999Z       -0.079505       -0.079504   
          2022-07-03T00:59:59.999Z       -0.079505       -0.079504   
          2022-07-04T00:59:59.999Z       -0.079505       -0.079504   
          2022-07-05T00:59:59.999Z       -0.079503       -0.079503   

                                    quote.USD.low  quote.USD.close  \
symbol_id time_close                                                 
MPH_7742  2020-11-23T00:59:59.999Z      -0.054870        -0.053279   
          2020-11-24T00:59:59.999Z      -0.054610        -0.054491   
          2020-11-25T00:59:59.999Z      -0.062961        -0.063061   
          2020-11-26T00:59:59.999Z      -0.067504        -0.067248   
          2020-11-27T00:59:59.999Z      -0.070376        -0.070215   
...                                           ...              ...   
CXO_2490  2022-07-01T00:59:59.999Z      -0.079534        -0.079513   
          2022-07-02T00:59:59.999Z      -0.079534        -0.079514   
          2022-07-03T00:59:59.999Z      -0.079534        -0.079514   
          2022-07-04T00:59:59.999Z      -0.079534        -0.079514   
          2022-07-05T00:59:59.999Z      -0.079533        -0.079513   

                                    quote.USD.volume  quote.USD.market_cap  \
symbol_id time_close                                                         
MPH_7742  2020-11-23T00:59:59.999Z         -0.002106             -0.059420   
          2020-11-24T00:59:59.999Z         -0.002107             -0.059420   
          2020-11-25T00:59:59.999Z         -0.002107             -0.059420   
          2020-11-26T00:59:59.999Z         -0.002108             -0.059420   
          2020-11-27T00:59:59.999Z         -0.002108             -0.059420   
...                                              ...                   ...   
CXO_2490  2022-07-01T00:59:59.999Z         -0.002108             -0.058753   
          2022-07-02T00:59:59.999Z         -0.002108             -0.058784   
          2022-07-03T00:59:59.999Z         -0.002108             -0.058783   
          2022-07-04T00:59:59.999Z         -0.002108             -0.058780   
          2022-07-05T00:59:59.999Z         -0.002108             -0.058738   

                                    minute_high  minute_low  sec_high  \
symbol_id time_close                                                    
MPH_7742  2020-11-23T00:59:59.999Z    -1.308405    0.420403 -0.056313   
          2020-11-24T00:59:59.999Z    -1.308405    0.564676 -0.056313   
          2020-11-25T00:59:59.999Z    -1.308405    1.430313 -0.167048   
          2020-11-26T00:59:59.999Z     1.395706   -0.733778 -0.277782   
          2020-11-27T00:59:59.999Z     1.443994   -0.589506 -0.277782   
...                                         ...         ...       ...   
CXO_2490  2022-07-01T00:59:59.999Z     0.333377   -1.022324 -0.831453   
          2022-07-02T00:59:59.999Z    -0.342651    1.478404 -0.831453   
          2022-07-03T00:59:59.999Z    -0.487514    0.324222 -0.831453   
          2022-07-04T00:59:59.999Z    -0.535802   -1.022324 -0.831453   
          2022-07-05T00:59:59.999Z     1.347418   -1.022324 -0.831453   

                                     sec_low  
symbol_id time_close                          
MPH_7742  2020-11-23T00:59:59.999Z -0.297594  
          2020-11-24T00:59:59.999Z -0.188209  
          2020-11-25T00:59:59.999Z -0.297594  
          2020

In [18]:
print(df_final_copy.shape)

(742019, 10)


In [ ]:
#dbscan
from sklearn.neighbors import NearestNeighbors # importing the library
neighb = NearestNeighbors(n_neighbors=10) # creating an object of the NearestNeighbors class
nbrs=neighb.fit(df_final_copy) # fitting the data to the object
distances,indices=nbrs.kneighbors(df_final_copy) # finding the nearest neighbours

In [ ]:
# Sort and plot the distances results
distances = np.sort(distances, axis = 0) # sorting the distances
distances = distances[:, 1] # taking the second column of the sorted distances
plt.rcParams['figure.figsize'] = (5,3) # setting the figure size
plt.plot(distances) # plotting the distances
plt.show() # showing the plot

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
DBSCAN_cluster = DBSCAN(eps=10, min_samples=5).fit(df_final_copy) 

In [ ]:
DBSCAN_cluster.labels_